In [1]:
import pandas as pd 
import numpy as np 
import lightgbm as lgb
from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from Dataloader import Dataloader
from RamdomForest import RamdomForest
from score import score
from sklearn.utils import shuffle
from IPython.display import display, HTML
from datetime import date
%matplotlib inline

In [2]:
df = pd.read_csv('data/train.csv')
df['avg_price'] = df['total_price']/df['building_area']

In [4]:

df.groupby('txn_floor').agg({'avg_price':['var','mean','sum','max','min','median','std']
                                            })

avg_price                                            \
                    var          mean           sum           max   
txn_floor                                                           
1.0        2.612890e+12  1.685963e+06  8.849621e+09  3.934795e+07   
2.0        1.459129e+12  1.349536e+06  7.419751e+09  1.280336e+07   
3.0        1.579301e+12  1.320726e+06  8.126428e+09  1.663299e+07   
4.0        1.487490e+12  1.297241e+06  8.315314e+09  1.220507e+07   
5.0        1.754845e+12  1.271003e+06  6.491014e+09  1.315998e+07   
6.0        2.163306e+12  1.338364e+06  3.716637e+09  1.723187e+07   
7.0        2.161402e+12  1.378635e+06  3.549986e+09  1.596310e+07   
8.0        1.663203e+12  1.245508e+06  2.205794e+09  1.261105e+07   
9.0        2.461545e+12  1.378044e+06  2.273773e+09  1.456143e+07   
10.0       2.306710e+12  1.323499e+06  1.982602e+09  1.405156e+07   
11.0       2.361064e+12  1.358527e+06  1.899220e+09  1.962925e+07   
12.0       2.009381e+12  1.267821e+06  1.513779e+09  1.520144e+07   
13.0       2.976619e+12  1.310386e+06  1.046999e+09  2.861616e+07   
14.0       1.185676e+12  1.147929e+06  7.737044e+08  8.126624e+06   
15.0       1.126725e+12  1.145225e+06  4.706873e+08  9.457596e+06   
16.0       8.514694e+11  1.093529e+06  2.580728e+08  6.488175e+06   
17.0       1.182541e+12  1.293776e+06  2.134731e+08  6.865858e+06   
18.0       8.198379e+11  1.322779e+06  1.931257e+08  6.318147e+06   
19.0       7.120994e+11  1.354606e+06  1.517158e+08  3.848385e+06   
20.0       6.153150e+11  1.277377e+06  9.452591e+07  3.614212e+06   
21.0       3.560104e+11  1.282950e+06  7.312814e+07  2.886451e+06   
22.0       4.216982e+11  1.176741e+06  5.295335e+07  2.921398e+06   
23.0       1.079400e+12  1.505697e+06  7.076777e+07  4.912034e+06   
24.0       6.047377e+11  1.459342e+06  2.626816e+07  3.056583e+06   
25.0       1.335912e+12  1.874992e+06  2.812487e+07  3.718737e+06   
26.0       2.773220e+11  1.179628e+06  1.061665e+07  2.391977e+06   
27.0       2.672532e+12  2.100743e+06  1.680595e+07  4.295554e+06   
28.0       8.338239e+11  1.371951e+06  2.743902e+06  2.017638e+06   

                                                      
                     min        median           std  
txn_floor                                             
1.0         54542.632662  1.415788e+06  1.616444e+06  
2.0         40270.010802  1.059753e+06  1.207944e+06  
3.0         57953.707722  1.021721e+06  1.256702e+06  
4.0         29638.472250  1.014305e+06  1.219627e+06  
5.0         48570.412561  9.364959e+05  1.324706e+06  
6.0         30492.139935  8.126147e+05  1.470818e+06  
7.0         57606.948559  8.833674e+05  1.470171e+06  
8.0         75455.888302  7.956550e+05  1.289652e+06  
9.0         76680.032271  8.281816e+05  1.568931e+06  
10.0        75455.888302  7.871173e+05  1.518786e+06  
11.0        89926.907155  8.252172e+05  1.536575e+06  
12.0        28537.167443  7.925591e+05  1.417527e+06  
13.0       101229.922166  7.954329e+05  1.725288e+06  
14.0        50766.178265  7.773606e+05  1.088888e+06  
15.0        79449.406907  8.159128e+05  1.061473e+06  
16.0        82637.113736  8.844533e+05  9.227510e+05  
17.0       178619.809045  1.034506e+06  1.087447e+06  
18.0       174793.170524  1.092385e+06  9.054490e+05  
19.0       146452.459204  1.262462e+06  8.438598e+05  
20.0       286065.743841  1.171071e+06  7.844202e+05  
21.0       217177.761910  1.253739e+06  5.966661e+05  
22.0       204159.623676  1.066150e+06  6.493829e+05  
23.0       309039.748069  1.284710e+06  1.038942e+06  
24.0       214264.104593  1.345643e+06  7.776488e+05  
25.0       232630.283199  1.398155e+06  1.155817e+06  
26.0       785417.818982  9.708166e+05  5.266137e+05  
27.0       453444.047151  1.447747e+06  1.634788e+06  
28.0       726263.908746  1.371951e+06  9.131396e+05

In [3]:

table = df.groupby('txn_floor').mean()['avg_price'].values
print(table)

[1685963.18229431 1349536.32765118 1320726.22194503 1297240.88435969
 1271003.35448145 1338363.88637557 1378635.31990427 1245507.63021452
 1378044.28948709 1323499.11260538 1358526.74139964 1267821.33862403
 1310386.3406763  1147929.44006086 1145224.51495315 1093528.81875352
 1293776.49272904 1322778.7919978  1354605.55928921 1277377.136509
 1282949.83169887 1176741.13182021 1505697.22305046 1459342.13033516
 1874991.51531772 1179627.93708831 2100743.33303832 1371951.08786529]


In [4]:
def foo(x):
    if np.isnan(x):
        return np.nan
    else:
        index = int(x)-1
        return table[index]
df['txn_floor_price'] = df['txn_floor'].apply(foo)

In [5]:
x = df.corr().apply(np.abs).sort_values('txn_floor_price', ascending=False)['txn_floor_price']

In [8]:
x.loc['avg_price']

0.09529963842181115